In [3]:
# Assume YahooDownloader and symbols are defined elsewhere
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.config import INDICATORS
import pandas as pd
import numpy as np
from datetime import datetime

/Users/zihan/miniconda3/envs/FinRL/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [18]:
symbols= [
    "aapl", "amd", "amzn", "f", "goog", "gs", "intc", "ko", "meta", "msft", "nflx", "nvda", "tsla", "v", "axp", "ba", "cat", "csco", "cvx", "dis", "dow", "hd", "hon", "ibm", "jnj", "jpm", "mcd", "mmm", "mrk", "nke", "pg", "trv", "unh"
]

START_DATE = '2012-05-18'
DATA_FILE = 'stock_data.csv'
END_DATE = datetime.today().strftime('%Y-%m-%d')

def fetch_and_update_data():
    print("Today's date:", END_DATE)

    try:
        existing_data = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        existing_data = pd.DataFrame()
    
    # Fetch new data
    new_data = YahooDownloader(start_date=START_DATE, end_date=END_DATE, ticker_list=symbols).fetch_data()

    # Check for overlaps
    if not existing_data.empty:
        last_date = existing_data['date'].max()
        new_data = new_data[new_data['date'] > last_date]

    print('Existing data:\n', existing_data)

    if not new_data.empty:
        updated_data = pd.concat([existing_data, new_data])
        updated_data.to_csv(DATA_FILE, index=False)
        print(f"Updated data up to {new_data['date'].max()}")
    

fetch_and_update_data()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Today's date: 2024-07-10



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

Shape of DataFrame:  (99031, 8)
Existing data:
 Empty DataFrame
Columns: []
Index: []
Updated data up to 2024-07-09


In [22]:
data = pd.read_csv(DATA_FILE)
data

,date,open,high,low,close,volume,tic,day
0,2012-05-18,19.070000,19.407499,18.649286,16.014688,732292400,aapl,4
1,2012-05-18,6.190000,6.290000,6.000000,6.010000,14979900,amd,4
2,2012-05-18,10.970500,10.981500,10.640500,10.692500,104634000,amzn,4
3,2012-05-18,55.740002,55.840000,55.049999,46.533321,8783700,axp,4
4,2012-05-18,69.949997,70.339996,68.930000,57.249622,5597400,ba,4
...,...,...,...,...,...,...,...,...
99026,2024-07-09,166.630005,166.759995,165.139999,165.660004,4393100,pg,1
99027,2024-07-09,203.759995,206.199997,202.830002,203.139999,936200,trv,1
99028,2024-07-09,251.000000,265.609985,250.300003,262.329987,160210900,tsla,1
99029,2024-07-09,488.329987,495.399994,486.089996,492.109985,3056800,unh,1


In [23]:
import itertools

fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(data)



list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))


processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
processed_full



Successfully added technical indicators


[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (3052, 8)
Successfully added vix
Successfully added turbulence index


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2012-05-18,aapl,19.070000,19.407499,18.649286,16.014688,732292400.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,16.014688,16.014688,25.10,0.000000
1,2012-05-18,amd,6.190000,6.290000,6.000000,6.010000,14979900.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,6.010000,6.010000,25.10,0.000000
2,2012-05-18,amzn,10.970500,10.981500,10.640500,10.692500,104634000.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,10.692500,10.692500,25.10,0.000000
3,2012-05-18,axp,55.740002,55.840000,55.049999,46.533321,8783700.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,46.533321,46.533321,25.10,0.000000
4,2012-05-18,ba,69.949997,70.339996,68.930000,57.249622,5597400.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,57.249622,57.249622,25.10,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141915,2024-07-08,pg,165.949997,166.669998,165.619995,166.520004,5144500.0,0.0,-0.130550,169.701646,163.171352,53.895704,20.193587,0.604437,165.920332,164.297979,12.37,34.302097
141916,2024-07-08,trv,202.869995,207.000000,202.690002,204.229996,852300.0,0.0,-2.297026,213.891159,200.624841,44.298112,-87.217745,14.062812,208.234828,211.528840,12.37,34.302097
141917,2024-07-08,tsla,247.710007,259.440002,244.570007,252.940002,157219600.0,0.0,17.209704,250.668235,144.346764,71.625311,251.870221,74.875611,190.564667,180.576333,12.37,34.302097
141918,2024-07-08,unh,489.190002,493.880005,486.170013,489.339996,3450600.0,0.0,-1.166691,503.292214,477.567405,48.777682,-35.505006,1.545014,492.862372,492.628579,12.37,34.302097


In [25]:
Processed_Stock_Data = 'Processed_Stock_Data.csv'
with open(Processed_Stock_Data, 'w', encoding = 'utf-8-sig') as f:
  processed_full.to_csv(f)

## DO not need

In [19]:
from datetime import datetime, timedelta

START_DATE = '2012-05-18'
END_DATE = datetime.today().strftime('%Y-%m-%d')

# Convert string dates to datetime objects
START_DATE = datetime.strptime(START_DATE, '%Y-%m-%d')
END_DATE = datetime.strptime(END_DATE, '%Y-%m-%d')

# Calculate the duration of the period
total_days = (END_DATE - START_DATE).days

# Calculate the split points
TRAIN_END_DATE = START_DATE + timedelta(days=(total_days * 2 // 3))
TRADE_START_DATE = START_DATE + timedelta(days=(total_days * 2 // 3))
TRADE_END_DATE = END_DATE

# Convert datetime objects back to string format if needed
TRAIN_START_DATE = START_DATE.strftime('%Y-%m-%d')
TRAIN_END_DATE = TRAIN_END_DATE.strftime('%Y-%m-%d')
TRADE_START_DATE = TRADE_START_DATE.strftime('%Y-%m-%d')
TRADE_END_DATE = TRADE_END_DATE.strftime('%Y-%m-%d')

In [20]:
TRAIN_START_DATE, TRAIN_END_DATE, TRADE_START_DATE, TRADE_END_DATE

('2012-05-18', '2020-06-22', '2020-06-22', '2024-07-10')

In [21]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

train_path = './train_data.csv'
trade_path = './trade_data.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)


65120
32544


In [24]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2012-05-18,aapl,19.070000,19.407499,18.649286,16.014688,732292400.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,16.014688,16.014688,25.100000,0.000000
0,2012-05-18,amd,6.190000,6.290000,6.000000,6.010000,14979900.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,6.010000,6.010000,25.100000,0.000000
0,2012-05-18,amzn,10.970500,10.981500,10.640500,10.692500,104634000.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,10.692500,10.692500,25.100000,0.000000
0,2012-05-18,axp,55.740002,55.840000,55.049999,46.533321,8783700.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,46.533321,46.533321,25.100000,0.000000
0,2012-05-18,ba,69.949997,70.339996,68.930000,57.249622,5597400.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,57.249622,57.249622,25.100000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,2020-06-19,pg,120.489998,121.820000,118.830002,107.711754,17506200.0,4.0,0.748190,109.751867,102.027789,52.631055,137.750221,8.897491,105.063030,104.950681,35.119999,16.148686
2034,2020-06-19,trv,117.750000,117.750000,115.250000,106.579033,4756900.0,4.0,3.667325,117.302586,90.293684,54.946250,66.508857,8.416846,97.673317,94.594931,35.119999,16.148686
2034,2020-06-19,tsla,67.518669,67.731331,66.089333,66.726669,130195500.0,4.0,3.783843,70.741875,51.277392,62.600840,122.737411,30.589977,58.669489,51.332033,35.119999,16.148686


In [1]:
import pandas as pd
train = pd.read_csv('./train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']

trade = pd.read_csv('./trade_data.csv')
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

In [4]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 32, State Space: 321


In [6]:
def process_df_for_mvo(df):
  df = df.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]
  fst = df
  fst = fst.iloc[0:stock_dimension, :]
  tic = fst['tic'].tolist()

  mvo = pd.DataFrame()

  for k in range(len(tic)):
    mvo[tic[k]] = 0

  for i in range(df.shape[0]//stock_dimension):
    n = df
    n = n.iloc[i * stock_dimension:(i+1) * stock_dimension, :]
    date = n['date'][i*stock_dimension]
    mvo.loc[date] = n['close'].tolist()
    
  return mvo


In [5]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [7]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

# TradeData.to_numpy()

In [11]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(StockData)
S = risk_models.sample_cov(StockData)

print(mu)
# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
# ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

aapl    0.230422
amd     0.313300
amzn    0.367534
axp     0.092925
ba      0.157966
cat     0.077253
csco    0.168853
cvx     0.029143
dis     0.141748
f      -0.011536
goog    0.214106
gs      0.114421
hd      0.256013
hon     0.158254
ibm    -0.023920
intc    0.141270
jnj     0.139680
jpm     0.173319
ko      0.060815
mcd     0.128073
meta    0.254791
mmm     0.111275
mrk     0.130118
msft    0.294299
nflx    0.604340
nke     0.187929
nvda    0.544106
pg      0.114630
trv     0.107441
tsla    0.560588
unh     0.252376
v       0.277661
dtype: float64
OrderedDict([('aapl', 0.0), ('amd', 0.0), ('amzn', 0.16744), ('axp', 0.0), ('ba', 0.0), ('cat', 0.0), ('csco', 0.0), ('cvx', 0.0), ('dis', 0.0), ('f', 0.0), ('goog', 0.0), ('gs', 0.0), ('hd', 0.12669), ('hon', 0.0), ('ibm', 0.0), ('intc', 0.0), ('jnj', 0.0), ('jpm', 0.0), ('ko', 0.0), ('mcd', 0.0), ('meta', 0.0), ('mmm', 0.0), ('mrk', 0.0), ('msft', 0.0), ('nflx', 0.2064), ('nke', 0.0), ('nvda', 0.24768), ('pg', 0.0), ('trv', 0.0), ('tsl

(0.4547287590770615, 0.257754771654041, 1.686598297627465)

## Start need

In [26]:
import pandas as pd

# Define constants
DATA_FILE = 'Processed_Stock_Data.csv'

# Sample input data
input_data = {
    'endDate': 742264650.932187,
    'basket': ['aapl', 'amd'],
    'IsChinese': False,
    'totalAmount': 1000,
    'riskFreeRate': 0,
    'startDate': 739672650.932178
}


In [35]:
from pypfopt.efficient_frontier import EfficientFrontier

# Function to run the model

def __init__(self):
    self.basket = input_data['basket']
    self.totalAmount = input_data['totalAmount']
    
    

# Function to get the stock data from the database based on the user request
def get_stock_data(basket): 
    raw_data = pd.read_csv(DATA_FILE)
    data = raw_data[(raw_data['tic'].isin(basket))]
    data = data.set_index(data.columns[0])
    data.index.names = ['']
    return data


def process_df_for_mvo(df, stock_dimension):
    df = df.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]
    fst = df
    fst = fst.iloc[0:stock_dimension, :]
    tic = fst['tic'].tolist()

    mvo = pd.DataFrame()

    for k in range(len(tic)):
        mvo[tic[k]] = 0

    for i in range(df.shape[0]//stock_dimension):
        n = df
        n = n.iloc[i * stock_dimension:(i+1) * stock_dimension, :]
        date = n['date'][i*stock_dimension]
        mvo.loc[date] = n['close'].tolist()
    return mvo
    
def results_calculation(raw_data, totalAmount, stock_dimension):
    data = process_df_for_mvo(raw_data, stock_dimension)

    # Calculate expected returns and sample covariance
    mu = expected_returns.mean_historical_return(data)
    S = risk_models.sample_cov(data)

    # Optimize for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))  ## Set the single asset can be 100% of the portfolio, but we can not buy short.
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    # ef.save_weights_to_file("weights.csv")  # saves to file
    mvo_weights = np.array([totalAmount * cleaned_weights[i] for i in range(len(cleaned_weights))]).tolist() # weight * the total amount of money for each stock
    report = ef.portfolio_performance(verbose=True)
    
    results = {
        'meanReturns': [f"{x * 100:.2f}%" for x in mu],
        'Allocated Weights': [f"{x:.2f}" for x in mvo_weights],
        'Expected annual return:': f"{report[0] * 100:.2f}%",
        'Annual volatility:': f"{report[1] * 100:.2f}%",
        'Sharpe Ratio:': f"{report[2]:.2f}"
    }
    
    return results

def run_model(raw_data, totalAmount):
    raw_data = get_stock_data(self.basket)
    ## Out-of-Sample Performance
    stock_dimension = len(raw_data.tic.unique())
    print(f"Stock Dimension: {stock_dimension}")
    # Call the results_calculation function
    results = results_calculation(raw_data, totalAmount, stock_dimension)
    return results

In [36]:
raw_data = get_stock_data(input_data['basket'])
raw_data

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2012-05-18,aapl,19.070000,19.407499,18.649286,16.014688,732292400.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,16.014688,16.014688,25.10,0.000000
1,2012-05-18,amd,6.190000,6.290000,6.000000,6.010000,14979900.0,4.0,0.000000,17.800680,15.161713,100.000000,66.666667,100.000000,6.010000,6.010000,25.10,0.000000
96,2012-05-21,aapl,19.089287,20.055000,19.073214,16.947704,631106000.0,0.0,0.020933,17.800680,15.161713,100.000000,66.666667,100.000000,16.481196,16.481196,22.01,0.000000
97,2012-05-21,amd,6.100000,6.340000,6.060000,6.300000,16378200.0,0.0,0.006506,6.565122,5.744878,100.000000,66.666667,100.000000,6.155000,6.155000,22.01,0.000000
128,2012-05-22,aapl,20.341070,20.495714,19.735001,16.817560,694870400.0,1.0,0.022637,17.603948,15.582688,87.389862,79.280535,100.000000,16.593318,16.593318,22.48,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141729,2024-07-03,amd,164.199997,165.259995,162.509995,163.899994,26980900.0,2.0,-0.030034,167.737634,154.417365,50.953985,33.387782,8.432472,162.415333,159.534167,12.09,22.416102
141792,2024-07-05,aapl,221.649994,226.449997,221.649994,226.339996,60412400.0,4.0,7.453463,227.983849,193.982152,69.703039,128.971776,57.740893,204.516333,190.721803,12.48,30.964845
141793,2024-07-05,amd,165.970001,174.039993,165.679993,171.899994,61857100.0,4.0,0.832292,169.314488,153.413510,55.037699,164.430605,25.845444,162.656666,159.552833,12.48,30.964845


In [37]:
model_results = run_model(raw_data, input_data['totalAmount'])

Stock Dimension: 2


KeyError: 0

In [46]:
model_results

{'meanReturns': ['9.76%', '17.89%'],
 'maxSharpeWeights': ['709.34', '290.66'],
 'Expected annual return:': '12.13%',
 'Annual volatility:': '190.26%',
 'Sharpe Ratio:': '0.05'}

In [39]:
import numpy as np
import pandas as pd
from pypfopt import risk_models, expected_returns, EfficientFrontier

DATA_FILE = 'Processed_Stock_Data.csv'

# Example input data
input_data = {
    'endDate': 742264650.932187,
    'basket': ['aapl', 'amd'],
    'IsChinese': False,
    'totalAmount': 1000,
    'riskFreeRate': 0,
    'startDate': 739672650.932178
}

class PortfolioModel:
    def __init__(self, input_data):
        self.basket = input_data['basket']
        self.totalAmount = input_data['totalAmount']

    # Function to get the stock data from the database based on the user request
    def get_stock_data(self):
        raw_data = pd.read_csv(DATA_FILE)
        data = raw_data[(raw_data['tic'].isin(self.basket))]
        data = data.set_index(data.columns[0])
        data.index.names = ['']
        return data

    def process_df_for_mvo(self, df, stock_dimension):
        df = df.sort_values(['date', 'tic'], ignore_index=True)[['date', 'tic', 'close']]
        fst = df.iloc[0:stock_dimension, :]
        tic = fst['tic'].tolist()

        mvo = pd.DataFrame()

        for k in range(len(tic)):
            mvo[tic[k]] = 0

        for i in range(df.shape[0] // stock_dimension):
            n = df.iloc[i * stock_dimension:(i + 1) * stock_dimension, :]
            date = n['date'].iloc[0]
            mvo.loc[date] = n['close'].tolist()
        return mvo

    def results_calculation(self, raw_data, stock_dimension):
        data = self.process_df_for_mvo(raw_data, stock_dimension)

        # Calculate expected returns and sample covariance
        mu = expected_returns.mean_historical_return(data)
        S = risk_models.sample_cov(data)

        # Optimize for maximal Sharpe ratio
        ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))  # Set the single asset can be 100% of the portfolio, but we cannot buy short.
        raw_weights = ef.max_sharpe()
        cleaned_weights = ef.clean_weights()
        mvo_weights = np.array([self.totalAmount * cleaned_weights[ticker] for ticker in cleaned_weights]).tolist()  # weight * the total amount of money for each stock
        report = ef.portfolio_performance(verbose=True)

        results = {
            'meanReturns': [f"{x * 100:.2f}%" for x in mu],
            'Allocated Weights': [f"{x:.2f}" for x in mvo_weights],
            'Expected annual return:': f"{report[0] * 100:.2f}%",
            'Annual volatility:': f"{report[1] * 100:.2f}%",
            'Sharpe Ratio:': f"{report[2]:.2f}"
        }

        return results

    def run_model(self):
        raw_data = self.get_stock_data()
        stock_dimension = len(raw_data.tic.unique())
        print(f"Number of Stocks = {stock_dimension}")
        results = self.results_calculation(raw_data, stock_dimension)
        return results

# Example usage
model = PortfolioModel(input_data)
results = model.run_model()
print(results)


Number of Stocks = 2
Expected annual return: 25.8%
Annual volatility: 28.5%
Sharpe Ratio: 0.83
{'meanReturns': ['24.52%', '32.34%'], 'Allocated Weights': ['841.12', '158.88'], 'Expected annual return:': '25.76%', 'Annual volatility:': '28.48%', 'Sharpe Ratio:': '0.83'}
